In [1]:
import pylab as pl
%matplotlib inline
pl.rcParams['figure.figsize'] = 15, 15

In [2]:
import os.path
import csv
import numpy as np
import scipy.spatial
import pandas as pd

In [3]:

os.chdir("prepared_corpora/msrp_ns_va_nophrase_mfcwsd/")

In [4]:
def embedding_distance(embeddings, id1, id2):
    ii = id1 - 1  #Change from 1 indexed id, to 0 indexex embedding index
    jj = id2 - 1
    return scipy.spatial.distance.cosine(embeddings[ii,:], embeddings[jj,:])

In [5]:

def load_links(link_filename):
    return np.loadtxt(link_filename, delimiter=",",skiprows=1, dtype=np.int)

def load_link_distances(link_filename, embeddings):
    links = load_links(link_filename)
    dists = pd.Series()
    for link in links:
        assert(len(link)==2)
        phrase_id = link[0]
        var_phrase_id = link[1]
        
        dists.loc[phrase_id] = embedding_distance(embeddings, phrase_id, var_phrase_id)
    return dists



In [6]:
def get_dists_table(embedding_filename, max_corruption = 10):
    embeddings = np.loadtxt(embedding_filename, delimiter=",")
    dists = pd.DataFrame()
    dists["paraphrase"] = load_link_distances("paraphrases.txt", embeddings)
    for corruption_level in range(1,max_corruption+1):
        link_filename = str(corruption_level)+"verb_anto_semantic_corruptions.txt"
        dists["verb_anto_" + str(corruption_level)] = load_link_distances(link_filename, embeddings)
        link_filename = str(corruption_level)+"noun_sym_semantic_corruptions.txt"
        dists["noun_sym_" + str(corruption_level)] = load_link_distances(link_filename, embeddings)
    return dists  
    

In [51]:
a=Counter(["aye", "bee","bee", "cee"])
b=Counter(["aye", "bee","bee", "see"])
keys = list(set(a.keys()).union(b.keys()))
avec = [a[key] for key in  keys]
avec

[2, 1, 1, 0]

In [54]:
def get_bow_dists_table(max_corruption = 10):
    from collections import Counter
    
    bows = [Counter(sent.split()) for sent in open("phrases.txt",'r')]
    
    def get_dist(linenum1, linenum2):
        from distance import jaccard
        bow1 = bows[linenum1-1]
        bow2 = bows[linenum2-1]
        keys = list(set(bow1.keys()).union(bow2.keys()))
        vec1 = [bow1[key] for key in  keys]
        vec2 = [bow2[key] for key in  keys]
        return scipy.spatial.distance.cosine(vec1,vec2)

    def get_dists(link_filename):
        dists = pd.Series()
        for link in load_links(link_filename):
            dists.loc[link[0]] = get_dist(link[0],link[1])
        return dists
        

    dists = pd.DataFrame()
    dists["paraphrase"] = get_dists("paraphrases.txt")
    for corruption_level in range(1,max_corruption+1):
        dists["verb_anto_" + str(corruption_level)] = get_dists(str(corruption_level)+"verb_anto_semantic_corruptions.txt")
        dists["noun_sym_" + str(corruption_level)] = get_dists(str(corruption_level)+"noun_sym_semantic_corruptions.txt")
    return dists  
    


In [27]:
def get_sow_jaccard_dists_table(max_corruption = 10):
    
    bows = [set(sent.split()) for sent in open("phrases.txt",'r')]
    
    def get_dist(linenum1, linenum2):
        from distance import jaccard
        s1 = bows[linenum1-1]
        s2 = bows[linenum2-1]
        return jaccard(s1,s2)

    def get_dists(link_filename):
        dists = pd.Series()
        for link in load_links(link_filename):
            dists.loc[link[0]] = get_dist(link[0],link[1])
        return dists
        

    dists = pd.DataFrame()
    dists["paraphrase"] = get_dists("paraphrases.txt")
    for corruption_level in range(1,max_corruption+1):
        dists["verb_anto_" + str(corruption_level)] = get_dists(str(corruption_level)+"verb_anto_semantic_corruptions.txt")
        dists["noun_sym_" + str(corruption_level)] = get_dists(str(corruption_level)+"noun_sym_semantic_corruptions.txt")
    return dists  
    


In [55]:
bow_dists = get_bow_dists_table()

/usr/local/lib/python3.4/site-packages/numpy/lib/npyio.py:823: UserWarning: loadtxt: Empty input file: "6verb_anto_semantic_corruptions.txt"
  warnings.warn('loadtxt: Empty input file: "%s"' % fname)
/usr/local/lib/python3.4/site-packages/numpy/lib/npyio.py:823: UserWarning: loadtxt: Empty input file: "7verb_anto_semantic_corruptions.txt"
  warnings.warn('loadtxt: Empty input file: "%s"' % fname)
/usr/local/lib/python3.4/site-packages/numpy/lib/npyio.py:823: UserWarning: loadtxt: Empty input file: "8verb_anto_semantic_corruptions.txt"
  warnings.warn('loadtxt: Empty input file: "%s"' % fname)
/usr/local/lib/python3.4/site-packages/numpy/lib/npyio.py:823: UserWarning: loadtxt: Empty input file: "9verb_anto_semantic_corruptions.txt"
  warnings.warn('loadtxt: Empty input file: "%s"' % fname)
/usr/local/lib/python3.4/site-packages/numpy/lib/npyio.py:823: UserWarning: loadtxt: Empty input file: "10verb_anto_semantic_corruptions.txt"
  warnings.warn('loadtxt: Empty input file: "%s"' % fname)

In [81]:
rae_dists = get_dists_table("outVectors_RAE2011.csv")
#wiki_doc2vec_dists = get_dists_table("outVectors_wiki_doc2vec.csv")
#hansard_doc2vec_dists = get_dists_table("outVectors_hansard_doc2vec.csv")
#wiki_sentence_doc2vec_dists = get_dists_table("outVectors_wiki_sentence_doc2vec.csv")
wiki_sentence_concat_doc2vec_dists = get_dists_table("outVectors_wiki_sentence_doc2vec.csv")
dbow_dists = get_dists_table("outVectors_wiki_sentence_model_dbow.csv")
#random_dists = get_dists_table("outVectors_random.csv")

/usr/local/lib/python3.4/site-packages/numpy/lib/npyio.py:823: UserWarning: loadtxt: Empty input file: "6verb_anto_semantic_corruptions.txt"
  warnings.warn('loadtxt: Empty input file: "%s"' % fname)
/usr/local/lib/python3.4/site-packages/numpy/lib/npyio.py:823: UserWarning: loadtxt: Empty input file: "7verb_anto_semantic_corruptions.txt"
  warnings.warn('loadtxt: Empty input file: "%s"' % fname)
/usr/local/lib/python3.4/site-packages/numpy/lib/npyio.py:823: UserWarning: loadtxt: Empty input file: "8verb_anto_semantic_corruptions.txt"
  warnings.warn('loadtxt: Empty input file: "%s"' % fname)
/usr/local/lib/python3.4/site-packages/numpy/lib/npyio.py:823: UserWarning: loadtxt: Empty input file: "9verb_anto_semantic_corruptions.txt"
  warnings.warn('loadtxt: Empty input file: "%s"' % fname)
/usr/local/lib/python3.4/site-packages/numpy/lib/npyio.py:823: UserWarning: loadtxt: Empty input file: "10verb_anto_semantic_corruptions.txt"
  warnings.warn('loadtxt: Empty input file: "%s"' % fname)

In [64]:
bow_dists.count()

paraphrase      7800
verb_anto_1     3061
noun_sym_1      7168
verb_anto_2      590
noun_sym_2      5503
verb_anto_3       82
noun_sym_3      3395
verb_anto_4       12
noun_sym_4      1747
verb_anto_5        2
noun_sym_5       813
verb_anto_6        0
noun_sym_6       309
verb_anto_7        0
noun_sym_7       104
verb_anto_8        0
noun_sym_8        22
verb_anto_9        0
noun_sym_9         9
verb_anto_10       0
noun_sym_10        0
dtype: int64

In [66]:
for nchanges in range(1,10+1):
    verb_antos = bow_dists["verb_anto_"+str(nchanges)]
    noun_syms =  bow_dists["noun_sym_"+str(nchanges)]
    valid_dists = bow_dists[np.logical_and(pd.notnull(verb_antos), pd.notnull(noun_syms))]
    print(nchanges, "\t", len(valid_dists))

1 	 2829
2 	 449
3 	 46
4 	 4
5 	 0
6 	 0
7 	 0
8 	 0
9 	 0
10 	 0


In [74]:
def n_rel_than_paraphrase(prefix, dists, operation =lambda para_dist,var_dist: var_dist<para_dist, max_changes=10):
    """
    prefix : eg "verb_anto_" or  "noun_sym_"
    """

    return [operation(dists.paraphrase, dists[prefix+str(nchanges)]).sum()/dists[prefix+str(nchanges)].count() 
                 for nchanges in range(1,max_changes+1)]

In [ ]:
get_results_table("verb_anto_", lambda para_dist,var_dist: var_dist>para_dist, 5,
                  URAE= rae_dists,
                  PVDM = wiki_sentence_concat_doc2vec_dists,
                  PVDBOW = dbow_dists,
                  BOW = bow_dists
                 )

In [113]:
def get_results_table(comparator_series_method, max_changes, **model_dists):
    
    res = pd.DataFrame()
    res["n_changes"] = list(range(1,max_changes+1))
    
    model_names = list(model_dists.keys())
    model_names.sort()
    model_names.reverse()
    for model_name in model_names:
        dists = model_dists[model_name]
        res[model_name] = comparator_series_method(dists,max_changes)
        #n_rel_than_paraphrase(prefix, dists, operation, max_changes)
    
    return res
        
    

In [118]:
pd.options.display.float_format = lambda x: '{:.0f}%'.format(x*100)
def noun_sym_lt_para(dists,max_changes):
    return n_rel_than_paraphrase("noun_sym_", dists, lambda para_dist,var_dist: var_dist<=para_dist, max_changes)
get_results_table(noun_sym_lt_para, 9,
                  URAE= rae_dists,
                  PVDM = wiki_sentence_concat_doc2vec_dists,
                  PVDBOW = dbow_dists,
                  BOW = bow_dists
                 )

,n_changes,URAE,PVDM,PVDBOW,BOW
0,1,95%,91%,90%,100%
1,2,91%,84%,82%,99%
2,3,87%,76%,75%,95%
3,4,83%,68%,67%,85%
4,5,79%,58%,57%,66%
5,6,75%,52%,50%,50%
6,7,75%,36%,39%,34%
7,8,73%,36%,27%,18%
8,9,89%,56%,33%,11%


In [119]:
pd.options.display.float_format = lambda x: '{:.0f}%'.format(x*100)

def verb_anto_gt_para(dists,max_changes):
    return n_rel_than_paraphrase("verb_anto_", dists, lambda para_dist,var_dist: var_dist>para_dist, max_changes)

get_results_table(verb_anto_gt_para, 5,
                  URAE= rae_dists,
                  PVDM = wiki_sentence_concat_doc2vec_dists,
                  PVDBOW = dbow_dists,
                  BOW = bow_dists
                 )

,n_changes,URAE,PVDM,PVDBOW,BOW
0,1,6%,8%,9%,0%
1,2,10%,13%,15%,1%
2,3,21%,22%,20%,1%
3,4,17%,17%,58%,0%
4,5,0%,0%,0%,100%


In [120]:
def anto_gt_sym(dists,max_changes):
    def inner():
        for nchanges in range(1,max_changes+1):
            verb_antos = dists["verb_anto_"+str(nchanges)]
            noun_syms =  dists["noun_sym_"+str(nchanges)]
            valid_dists = dists[np.logical_and(pd.notnull(verb_antos), pd.notnull(noun_syms))]
            yield (verb_antos > noun_syms).sum()/len(valid_dists)
    return list(inner())

get_results_table(anto_gt_sym, 4,
                  URAE= rae_dists,
                  PVDM = wiki_sentence_concat_doc2vec_dists,
                  PVDBOW = dbow_dists,
                  BOW = bow_dists
                 )

,n_changes,URAE,PVDM,PVDBOW,BOW
0,1,61%,49%,50%,1%
1,2,58%,50%,43%,4%
2,3,43%,39%,43%,7%
3,4,0%,25%,75%,25%


In [128]:
def anto_eq_sym(dists,max_changes):
    def inner():
        for nchanges in range(1,max_changes+1):
            verb_antos = dists["verb_anto_"+str(nchanges)]
            noun_syms =  dists["noun_sym_"+str(nchanges)]
            valid_dists = dists[np.logical_and(pd.notnull(verb_antos), pd.notnull(noun_syms))]
            yield (verb_antos == noun_syms).sum()/len(valid_dists)
    return list(inner())

get_results_table(anto_eq_sym, 4,
                  URAE= rae_dists,
                  PVDM = wiki_sentence_concat_doc2vec_dists,
                  PVDBOW = dbow_dists,
                  BOW = bow_dists
                 )

,n_changes,URAE,PVDM,PVDBOW,BOW
0,1,0%,0%,0%,95%
1,2,0%,0%,0%,87%
2,3,0%,0%,0%,87%
3,4,0%,0%,0%,75%


In [77]:


def get_comparative_distances_table(dists, max_changes = 10):
    def n_closer_than_paraphrase(prefix):
        """
        prefix : eg "verb_anto_" or  "noun_sym_"
        """
        return n_rel_than_paraphrase(prefix, dists, lambda para_dist,var_dist: var_dist<para_dist, max_changes)


    def compare_sym_anto_distances():
        def inner():
            for nchanges in range(1,max_changes+1):
                verb_antos = dists["verb_anto_"+str(nchanges)]
                noun_syms =  dists["noun_sym_"+str(nchanges)]
                valid_dists = dists[np.logical_and(pd.notnull(verb_antos), pd.notnull(noun_syms))]
                yield (verb_antos > noun_syms).sum()/len(valid_dists)
        return list(inner())
    
    comparative_distances = pd.DataFrame()
    comparative_distances["n_changes"] = list(range(1,max_changes+1))
    comparative_distances["noun_sym_lt_para"] = n_closer_than_paraphrase("noun_sym_")
    comparative_distances["verb_anto_lt_para"] = n_closer_than_paraphrase("verb_anto_")
    comparative_distances["noun_sym_lt_verb_anto"] = compare_sym_anto_distances()
    return comparative_distances
    
    
    



In [78]:
get_comparative_distances_table(bow_dists)

,n_changes,noun_sym_lt_para,verb_anto_lt_para,noun_sym_lt_verb_anto
0,1,0.997628,0.996733,0.009544
1,2,0.987643,0.984746,0.044543
2,3,0.947570,0.939024,0.065217
3,4,0.832856,1.000000,0.250000
4,5,0.654367,0.000000,NaN
5,6,0.491909,NaN,NaN
6,7,0.317308,NaN,NaN
7,8,0.090909,NaN,NaN
8,9,0.111111,NaN,NaN
9,10,NaN,NaN,NaN


In [11]:
get_comparative_distances_table(wiki_sentence_doc2vec_dists)

,n_changes,noun_sym_lt_para,verb_anto_lt_para,noun_sym_lt_verb_anto
0,1,0.913365,0.917674,0.493107
1,2,0.839179,0.866102,0.498886
2,3,0.761414,0.780488,0.391304
3,4,0.675444,0.833333,0.250000
4,5,0.584256,1.000000,NaN
5,6,0.517799,NaN,NaN
6,7,0.355769,NaN,NaN
7,8,0.363636,NaN,NaN
8,9,0.555556,NaN,NaN
9,10,NaN,NaN,NaN


In [12]:
get_comparative_distances_table(wiki_sentence_concat_doc2vec_dists)

,n_changes,noun_sym_lt_para,verb_anto_lt_para,noun_sym_lt_verb_anto
0,1,0.913365,0.917674,0.493107
1,2,0.839179,0.866102,0.498886
2,3,0.761414,0.780488,0.391304
3,4,0.675444,0.833333,0.250000
4,5,0.584256,1.000000,NaN
5,6,0.517799,NaN,NaN
6,7,0.355769,NaN,NaN
7,8,0.363636,NaN,NaN
8,9,0.555556,NaN,NaN
9,10,NaN,NaN,NaN


In [13]:
get_comparative_distances_table(rae_dists)

,n_changes,noun_sym_lt_para,verb_anto_lt_para,noun_sym_lt_verb_anto
0,1,0.949637,0.936949,0.607635
1,2,0.905506,0.896610,0.581292
2,3,0.866274,0.792683,0.434783
3,4,0.827132,0.833333,0.000000
4,5,0.788438,1.000000,NaN
5,6,0.754045,NaN,NaN
6,7,0.750000,NaN,NaN
7,8,0.727273,NaN,NaN
8,9,0.888889,NaN,NaN
9,10,NaN,NaN,NaN


In [ ]:
get_comparative_distances_table(wiki_doc2vec_dists)

In [ ]:
get_comparative_distances_table(hansard_doc2vec_dists)

In [ ]:
comparative_distances["n_changes"] = list(range(1,11))
#comparative_distances["RAE_noun_sym_lt_para"] = n_closer_than_paraphrase("noun_sym_", rae_dists)
#comparative_distances["RAE_verb_anto_lt_para"] = n_closer_than_paraphrase("verb_anto_", rae_dists)
comparative_distances["wiki_doc2vec_noun_sym_lt_para"] = n_closer_than_paraphrase("noun_sym_", wiki_doc2vec_dists)
comparative_distances["wiki_doc2vec_verb_anto_lt_para"] = n_closer_than_paraphrase("verb_anto_", wiki_doc2vec_dists)
#comparative_distances["hansard_doc2vec_noun_sym_lt_para"] = n_closer_than_paraphrase("noun_sym_", hansard_doc2vec_dists)
#comparative_distances["hansard_doc2vec_verb_anto_lt_para"] = n_closer_than_paraphrase("verb_anto_", hansard_doc2vec_dists)
comparative_distances

comparative_distances.loc[0:max_changes-2,"rae_noun_sym_lt_verb_anto"] = compare_sym_anto_distances(rae_dists,max_changes)
comparative_distances.loc[0:max_changes-2,"wiki_doc2vec_noun_sym_lt_verb_anto"] = compare_sym_anto_distances(wiki_doc2vec_dists,max_changes)
comparative_distances.loc[0:max_changes-2,"hansard_doc2vec_noun_sym_lt_verb_anto"] = compare_sym_anto_distances(hansard_doc2vec_dists,max_changes)
comparative_distances

In [ ]:
def drop_null_cols(df):
    keep_cols = [col for col in df.columns if not(all(pd.isnull(df.loc[:,col])))]
    return df.loc[:,keep_cols]
    


In [ ]:
drop_null_cols(hansard_doc2vec_dists).hist(sharex=True)

In [ ]:
drop_null_cols(wiki_doc2vec_dists).hist(sharex=True)

In [ ]:
drop_null_cols(rae_dists).hist(sharex=True)

In [ ]:
!git push